### Import necessary libraries and modules

In [1]:
# %%bash

# kaggle datasets download -d nageshsingh/dna-sequence-dataset
# mkdir data data/DNA
# unzip dna-sequence-dataset.zip -d data/DNA
# rm dna-sequence-dataset.zip

In [2]:
import os
import pickle
import time
from collections import OrderedDict
from typing import (
    List, Tuple, Dict, Optional, Callable, Union
)

import numpy as np
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import flwr as fl
from flwr.common import (
    Metrics, EvaluateIns, EvaluateRes, FitIns, FitRes, MetricsAggregationFn, 
    Scalar, logger, ndarrays_to_parameters, parameters_to_ndarrays,
    Parameters, NDArrays
)

from flwr.server.client_proxy import ClientProxy
from flwr.server.client_manager import ClientManager
from flwr.server.strategy.aggregate import weighted_loss_avg, aggregate
from logging import WARNING

from utils import *

### Model Architecture Creation

In [3]:
class Net(nn.Module):
    """
    Args:
        num_classes: An integer indicating the number of classes in the dataset.
    """
    def __init__(self, num_classes=10) -> None:
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_sp, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 8)
        self.fc5 = nn.Linear(8, num_classes)        

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the neural network
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))        
        x = self.fc5(x)
        return x

### Define the FlowerClient class for federated learning

In [4]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, device, batch_size, save_results, matrix_path, roc_path,
                 yaml_path, classes):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.cid = cid
        self.device = device
        self.batch_size = batch_size
        self.save_results = save_results
        self.matrix_path = matrix_path
        self.roc_path = roc_path
        self.yaml_path = yaml_path
        self.classes = classes

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters2(self.net)

    def fit(self, parameters, config):
        server_round = config['server_round']
        local_epochs = config['local_epochs']
        lr = float(config["learning_rate"])

        print(f'[Client {self.cid}, round {server_round}] fit, config: {config}')

        set_parameters(self.net, parameters)

        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.net.parameters(), lr=lr)

        results = engine.train(self.net, self.trainloader, self.valloader, optimizer=optimizer, loss_fn=criterion,
                               epochs=local_epochs, device=self.device)

        if self.save_results:
            save_graphs(self.save_results, local_epochs, results, f"_Client {self.cid}")

        return get_parameters2(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)

        loss, accuracy, y_pred, y_true, y_proba = engine.test(self.net, self.valloader,
                                                              loss_fn=torch.nn.CrossEntropyLoss(), device=self.device)

        if self.save_results:
            os.makedirs(self.save_results, exist_ok=True)
            if self.matrix_path:
                save_matrix(y_true, y_pred, self.save_results + self.matrix_path, self.classes)
            if self.roc_path:
                save_roc(y_true, y_proba, self.save_results + self.roc_path, len(self.classes))

        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

### Define the client_common function to set up the Flower client

In [5]:
def client_common(cid, model_save, path_yaml, path_roc, results_save, path_matrix,
                  batch_size, trainloaders, valloaders, DEVICE, CLASSES):
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    net = Net(num_classes=len(CLASSES)).to(DEVICE)

    if os.path.exists(model_save):
        print(" To get the checkpoint")
        checkpoint = torch.load(model_save, map_location=DEVICE)['model_state_dict']
        net.load_state_dict(checkpoint)

    return FlowerClient(cid, net, trainloader, valloader, device=DEVICE, batch_size=batch_size,
                        matrix_path=path_matrix, roc_path=path_roc, save_results=results_save, yaml_path=path_yaml,
                        classes=CLASSES)

### Define utility functions for federated learning

In [6]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    return {"accuracy": sum(accuracies) / sum(examples)}

def evaluate2(server_round: int, parameters: NDArrays,
              config: Dict[str, Scalar]) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    set_parameters(central, parameters)
    loss, accuracy, y_pred, y_true, y_proba = engine.test(central, testloader, loss_fn=torch.nn.CrossEntropyLoss(),
                                                          device=DEVICE)
    os.makedirs(save_results, exist_ok=True)
    save_matrix(y_true, y_pred, save_results + "confusion_matrix_test.png", CLASSES)
    save_roc(y_true, y_proba, save_results + "roc_test.png", len(CLASSES))
    
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

def get_on_fit_config_fn(epoch=2, lr=0.001, batch_size=32) -> Callable[[int], Dict[str, str]]:
    def fit_config(server_round: int) -> Dict[str, str]:
        config = {
            "learning_rate": str(lr),
            "batch_size": str(batch_size),
            "server_round": server_round,
            "local_epochs": epoch
        }
        return config
    return fit_config

def aggreg_fit_checkpoint(server_round, aggregated_parameters, central_model, path_checkpoint):
    if aggregated_parameters is not None:
        print(f"Saving round {server_round} aggregated_parameters...")
        aggregated_ndarrays: List[np.ndarray] = parameters_to_ndarrays(aggregated_parameters)
        
        params_dict = zip(central_model.state_dict().keys(), aggregated_ndarrays)
        state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
        central_model.load_state_dict(state_dict, strict=True)
        if path_checkpoint:
            torch.save({
                'model_state_dict': central_model.state_dict(),
            }, path_checkpoint)

### Define the FedCustom strategy class

In [7]:
# A Strategy from scratch with the same sampling of the clients as it is in FedAvg
# and then change the configuration dictionary
class FedCustom(fl.server.strategy.Strategy):
    def __init__(
            self,
            fraction_fit: float = 1.0,
            fraction_evaluate: float = 1.0,
            min_fit_clients: int = 2,
            min_evaluate_clients: int = 2,
            min_available_clients: int = 2,
            evaluate_fn: Optional[
                    Callable[[int, NDArrays, Dict[str, Scalar]], Optional[Tuple[float, Dict[str, Scalar]]]]
                ] = None,
            on_fit_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
            on_evaluate_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
            accept_failures: bool = True,
            initial_parameters: Optional[Parameters] = None,
            fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
            evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn,
        self.accept_failures = accept_failures
        self.initial_parameters = initial_parameters
        self.fit_metrics_aggregation_fn = fit_metrics_aggregation_fn
        self.evaluate_metrics_aggregation_fn = evaluate_metrics_aggregation_fn

    def __repr__(self) -> str:
        # Same function as FedAvg(Strategy)
        return f"FedCustom (accept_failures={self.accept_failures})"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        # Same function as FedAvg(Strategy)
        initial_parameters = self.initial_parameters
        self.initial_parameters = None  # Don't keep initial parameters in memory
        return initial_parameters

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        # Same function as FedAvg(Strategy)
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )

        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
        # Create custom configs
        n_clients = len(clients)
        half_clients = n_clients // 2
        # Custom fit config function provided
        standard_lr = lr
        higher_lr = 0.003
        config = {"server_round": server_round, "local_epochs": 1}
        if self.on_fit_config_fn is not None:
            # Custom fit config function provided
            config = self.on_fit_config_fn(server_round)

        # fit_ins = FitIns(parameters, config)
        # Return client/config pairs
        fit_configurations = []
        for idx, client in enumerate(clients):
            config["learning_rate"] = standard_lr if idx < half_clients else higher_lr
            """
            Each pair of (ClientProxy, FitRes) constitutes 
            a successful update from one of the previously selected clients.
            """
            fit_configurations.append(
                (
                    client,
                    FitIns(
                        parameters,
                        config
                    )
                )
            )
        # Successful updates from the previously selected and configured clients
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average. (each round)"""
        # Same function as FedAvg(Strategy)
        if not results:
            return None, {}

        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # Convert results parameters --> array matrix
        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]

        # Aggregate parameters using weighted average between the clients and convert back to parameters object (bytes)
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))

        metrics_aggregated = {}
        # Aggregate custom metrics if aggregation fn was provided
        if self.fit_metrics_aggregation_fn:
            fit_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)

        elif server_round == 1:  # Only log this warning once
            logger.log(WARNING, "No fit_metrics_aggregation_fn provided")

        # Same function as SaveModelStrategy(fl.server.strategy.FedAvg)
        """Aggregate model weights using weighted average and store checkpoint"""
        aggreg_fit_checkpoint(server_round, parameters_aggregated, central, model_save)
        return parameters_aggregated, metrics_aggregated

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        # Same function as FedAvg(Strategy)
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        # Same function as FedAvg(Strategy)
        # Do not configure federated evaluation if fraction eval is 0.
        if self.fraction_evaluate == 0.0:
            return []

        # Parameters and config
        config = {}  # {"server_round": server_round, "local_epochs": 1}

        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )

        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        # Each pair of (ClientProxy, FitRes) constitutes a successful update from one of the previously selected clients
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""
        # Same function as FedAvg(Strategy)
        if not results:
            return None, {}

        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # Aggregate loss
        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )

        metrics_aggregated = {}
        # Aggregate custom metrics if aggregation fn was provided
        if self.evaluate_metrics_aggregation_fn:
            eval_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.evaluate_metrics_aggregation_fn(eval_metrics)

        # Only log this warning once
        elif server_round == 1:
            logger.log(WARNING, "No evaluate_metrics_aggregation_fn provided")

        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""
        # Same function as FedAvg(Strategy)
        if self.evaluate_fn is None:
            # Let's assume we won't perform the global model evaluation on the server side.
            return None

        # if we have a global model evaluation on the server side :
        parameters_ndarrays = parameters_to_ndarrays(parameters)
        eval_res = self.evaluate_fn(server_round, parameters_ndarrays, {})

        # if you haven't results
        if eval_res is None:
            return None

        loss, metrics = eval_res
        return loss, metrics

### Set up the federated learning strategy

In [8]:
# Set up your variables directly
data_path = 'data/'
dataset = 'DNA'
yaml_path = './results/FL_DNA/results.yml'
seed = 0
num_workers = 0
max_epochs = 10
batch_size = 16
splitter = 10
device = 'gpu'
number_clients = 10
save_results = 'results/FL_DNA/'
matrix_path = 'confusion_matrix.png'
roc_path = 'roc.png'
model_save = 'results/FL_DNA/DNA_fl.pt'
min_fit_clients = 10
min_avail_clients = 10
min_eval_clients = 10
rounds = 20
frac_fit = 1.0
frac_eval = 0.5
lr = 1e-3

In [9]:
DEVICE = torch.device(choice_device(device))
CLASSES = classes_string(dataset)
trainloaders, valloaders, testloader = data_setup.load_datasets(num_clients=number_clients, batch_size=batch_size, resize=None, seed=seed, num_workers=num_workers, splitter=splitter, dataset=dataset, data_path=data_path, data_path_val=None)
_, input_sp = next(iter(testloader))[0].shape
central = Net(num_classes=len(CLASSES)).to(DEVICE)

DNA
The training set is created for the classes: ('0', '1', '2', '3', '4', '5', '6')


In [10]:
strategy = FedCustom(
    fraction_fit=frac_fit,
    fraction_evaluate=frac_eval,
    min_fit_clients=min_fit_clients,
    min_evaluate_clients=min_eval_clients if min_eval_clients else number_clients // 2,
    min_available_clients=min_avail_clients,
    evaluate_metrics_aggregation_fn=weighted_average,
    initial_parameters=ndarrays_to_parameters(get_parameters2(central)),
    evaluate_fn=evaluate2,
    on_fit_config_fn=get_on_fit_config_fn(epoch=max_epochs, batch_size=batch_size),
)

In [11]:
trainloaders, valloaders, testloader = data_setup.load_datasets(num_clients=number_clients,
                                                                batch_size=batch_size,
                                                                resize=None,
                                                                seed=seed,
                                                                num_workers=num_workers,
                                                                splitter=splitter,
                                                                dataset=dataset,  # Use the specified dataset
                                                                data_path=data_path,
                                                                data_path_val=None)  # Use the same path for validation data

def client_fn(cid: str) -> FlowerClient:
    return client_common(cid,
                         model_save, path_yaml, path_roc, results_save, path_matrix,
                         batch_size, trainloaders, valloaders, DEVICE, CLASSES)

DNA
The training set is created for the classes: ('0', '1', '2', '3', '4', '5', '6')


### Define the client_fn function and set up the simulation

In [12]:
import warnings
warnings.simplefilter("ignore")

print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)
print(f"Training on {DEVICE}")

client_resources = {"num_cpus": 11}

if DEVICE.type == "cuda":
    client_resources["num_gpus"] = 1

model_save = model_save
path_yaml = yaml_path
path_roc = roc_path
results_save = save_results
path_matrix = matrix_path
batch_size = batch_size

print("Start simulation")
start_simulation = time.time()
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=number_clients,
    config=fl.server.ServerConfig(num_rounds=rounds),
    strategy=strategy,
    client_resources=client_resources
)
print(f"Simulation Time = {time.time() - start_simulation} seconds")

INFO flwr 2024-10-06 18:41:22,649 | app.py:175 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)


flwr 1.5.0
numpy 1.26.4
torch 2.4.0+cu121
torchvision 0.19.0+cu121
Training on cuda:0
Start simulation


2024-10-06 18:41:23,660	WARNING utils.py:592 -- Ray currently does not support initializing Ray with fractional cpus. Your num_cpus will be truncated from 11.5 to 11.
2024-10-06 18:41:24,851	INFO worker.py:1786 -- Started a local Ray instance.
INFO flwr 2024-10-06 18:41:25,431 | app.py:210 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'node:10.42.22.191': 1.0, 'memory': 159472843367.0, 'object_store_memory': 72631218585.0, 'node:__internal_head__': 1.0, 'accelerator_type:A100': 1.0, 'CPU': 11.0}
INFO flwr 2024-10-06 18:41:25,432 | app.py:224 | Flower VCE: Resources for each Virtual Client: {'num_cpus': 11, 'num_gpus': 1}
INFO flwr 2024-10-06 18:41:25,445 | app.py:270 | Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO flwr 2024-10-06 18:41:25,445 | server.py:89 | Initializing global parameters
INFO flwr 2024-10-06 18:41:25,446 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2024-10-06 18:41:25,446 | server.py:91 | Evaluating ini

Updated model


INFO flwr 2024-10-06 18:41:28,672 | server.py:94 | initial parameters (loss, other metrics): 1.8805802258578215, {'accuracy': 31.363636363636367}
INFO flwr 2024-10-06 18:41:28,673 | server.py:104 | FL starting
DEBUG flwr 2024-10-06 18:41:28,673 | server.py:222 | fit_round 1: strategy sampled 10 clients (out of 10)


...

(DefaultActor pid=46681)  To get the checkpoint
(DefaultActor pid=46681) [Client 3] evaluate, config: {}
(DefaultActor pid=46681) Updated model


(DefaultActor pid=46681) /tmp/ipykernel_46150/3044491048.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
(DefaultActor pid=46681) /home/cudaq/.local/lib/python3.10/site-p

(DefaultActor pid=46681)  To get the checkpoint
(DefaultActor pid=46681) [Client 5] evaluate, config: {}
(DefaultActor pid=46681) Updated model


(DefaultActor pid=46681) /tmp/ipykernel_46150/3044491048.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
DEBUG flwr 2024-10-06 19:33:49,096 | server.py:187 | evaluate_rou

Simulation Time = 3146.4697701931 seconds
